In [67]:
import sqlite3 as sql
import pandas as pd
import numpy as np
import json

In [68]:
# Connect to database file
my_database = "../data/raw_data/chinook.db"
connection = sql.connect(database=my_database)
cursor = connection.cursor()

In [69]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
tables

[('albums',),
 ('sqlite_sequence',),
 ('artists',),
 ('customers',),
 ('employees',),
 ('genres',),
 ('invoices',),
 ('invoice_items',),
 ('media_types',),
 ('playlists',),
 ('playlist_track',),
 ('tracks',),
 ('sqlite_stat1',)]

In [70]:
cursor.execute("SELECT * FROM artists;")
artists = cursor.fetchall()
artists

[(1, 'AC/DC'),
 (2, 'Accept'),
 (3, 'Aerosmith'),
 (4, 'Alanis Morissette'),
 (5, 'Alice In Chains'),
 (6, 'Antônio Carlos Jobim'),
 (7, 'Apocalyptica'),
 (8, 'Audioslave'),
 (9, 'BackBeat'),
 (10, 'Billy Cobham'),
 (11, 'Black Label Society'),
 (12, 'Black Sabbath'),
 (13, 'Body Count'),
 (14, 'Bruce Dickinson'),
 (15, 'Buddy Guy'),
 (16, 'Caetano Veloso'),
 (17, 'Chico Buarque'),
 (18, 'Chico Science & Nação Zumbi'),
 (19, 'Cidade Negra'),
 (20, 'Cláudio Zoli'),
 (21, 'Various Artists'),
 (22, 'Led Zeppelin'),
 (23, 'Frank Zappa & Captain Beefheart'),
 (24, 'Marcos Valle'),
 (25, 'Milton Nascimento & Bebeto'),
 (26, 'Azymuth'),
 (27, 'Gilberto Gil'),
 (28, 'João Gilberto'),
 (29, 'Bebel Gilberto'),
 (30, 'Jorge Vercilo'),
 (31, 'Baby Consuelo'),
 (32, 'Ney Matogrosso'),
 (33, 'Luiz Melodia'),
 (34, 'Nando Reis'),
 (35, 'Pedro Luís & A Parede'),
 (36, 'O Rappa'),
 (37, 'Ed Motta'),
 (38, 'Banda Black Rio'),
 (39, 'Fernanda Porto'),
 (40, 'Os Cariocas'),
 (41, 'Elis Regina'),
 (42, 'Mi

In [71]:
cursor.execute("SELECT artists.*, COUNT(albums.ArtistId) AS album_count FROM artists LEFT JOIN albums ON artists.ArtistId = albums.ArtistId GROUP BY artists.ArtistId ORDER BY CASE WHEN artists.Name = 'Various Artists' THEN 1 ELSE 0 END, album_count DESC;")
artists_most_albuns = cursor.fetchall()
type(artists_most_albuns)

list

In [72]:
top_ten = artists_most_albuns[:10]

top_ten

[(90, 'Iron Maiden', 21),
 (22, 'Led Zeppelin', 14),
 (58, 'Deep Purple', 11),
 (50, 'Metallica', 10),
 (150, 'U2', 10),
 (114, 'Ozzy Osbourne', 6),
 (118, 'Pearl Jam', 5),
 (82, 'Faith No More', 4),
 (84, 'Foo Fighters', 4),
 (149, 'Lost', 4)]

In [73]:
json_data = {"artists": []}

for artist in top_ten:
    json_data["artists"].append({
        "artist_id": artist[0],  # Assuming the artist_id is at index 0
        "name": artist[1],       # Assuming the name is at index 1
        "album_count": artist[2] # Assuming the album_count is at index 2
    })

In [74]:
path_10_artists = "../data/processed_data/10_most_colaborative_artists.json"

json_data

with open(path_10_artists, "w") as file:
    json.dump(json_data, file, indent=2)

#Part 2

In [75]:
cursor.execute("SELECT * FROM genres;")
genres = cursor.fetchall()
genres

[(1, 'Rock'),
 (2, 'Jazz'),
 (3, 'Metal'),
 (4, 'Alternative & Punk'),
 (5, 'Rock And Roll'),
 (6, 'Blues'),
 (7, 'Latin'),
 (8, 'Reggae'),
 (9, 'Pop'),
 (10, 'Soundtrack'),
 (11, 'Bossa Nova'),
 (12, 'Easy Listening'),
 (13, 'Heavy Metal'),
 (14, 'R&B/Soul'),
 (15, 'Electronica/Dance'),
 (16, 'World'),
 (17, 'Hip Hop/Rap'),
 (18, 'Science Fiction'),
 (19, 'TV Shows'),
 (20, 'Sci Fi & Fantasy'),
 (21, 'Drama'),
 (22, 'Comedy'),
 (23, 'Alternative'),
 (24, 'Classical'),
 (25, 'Opera')]

In [76]:
query = "SELECT genres.Name AS genre_name, COUNT(tracks.TrackId) AS track_count FROM genres JOIN tracks ON genres.GenreId = tracks.GenreId GROUP BY genres.GenreId, genre_name ORDER BY track_count DESC;"

cursor.execute(query)

top_5_genres = cursor.fetchall()
top_5_genres

[('Rock', 1297),
 ('Latin', 579),
 ('Metal', 374),
 ('Alternative & Punk', 332),
 ('Jazz', 130),
 ('TV Shows', 93),
 ('Blues', 81),
 ('Classical', 74),
 ('Drama', 64),
 ('R&B/Soul', 61),
 ('Reggae', 58),
 ('Pop', 48),
 ('Soundtrack', 43),
 ('Alternative', 40),
 ('Hip Hop/Rap', 35),
 ('Electronica/Dance', 30),
 ('Heavy Metal', 28),
 ('World', 28),
 ('Sci Fi & Fantasy', 26),
 ('Easy Listening', 24),
 ('Comedy', 17),
 ('Bossa Nova', 15),
 ('Science Fiction', 13),
 ('Rock And Roll', 12),
 ('Opera', 1)]

In [77]:
genre_names = [genre[0] for genre in top_5_genres]

print(genre_names)
selected_genres_tuple = tuple(genre_names)

['Rock', 'Latin', 'Metal', 'Alternative & Punk', 'Jazz', 'TV Shows', 'Blues', 'Classical', 'Drama', 'R&B/Soul', 'Reggae', 'Pop', 'Soundtrack', 'Alternative', 'Hip Hop/Rap', 'Electronica/Dance', 'Heavy Metal', 'World', 'Sci Fi & Fantasy', 'Easy Listening', 'Comedy', 'Bossa Nova', 'Science Fiction', 'Rock And Roll', 'Opera']


In [78]:
query_num_artists = f"""
    SELECT
        genres.Name AS genre_name,
        COUNT(DISTINCT artists.ArtistId) AS num_artists
    FROM
        genres
    LEFT JOIN
        tracks ON genres.GenreId = tracks.GenreId
    LEFT JOIN
        albums ON tracks.AlbumId = albums.AlbumId
    LEFT JOIN
        artists ON albums.ArtistId = artists.ArtistId
    WHERE
        genres.Name IN {selected_genres_tuple}
    GROUP BY
        genres.GenreId, genre_name
    ORDER BY
        num_artists DESC;
"""


cursor.execute(query_num_artists)

number_of_artists = cursor.fetchall()
number_of_artists

[('Classical', 66),
 ('Rock', 51),
 ('Latin', 28),
 ('Alternative & Punk', 16),
 ('Metal', 14),
 ('Jazz', 10),
 ('Blues', 5),
 ('TV Shows', 5),
 ('Alternative', 5),
 ('Soundtrack', 4),
 ('R&B/Soul', 4),
 ('Reggae', 3),
 ('Pop', 3),
 ('Electronica/Dance', 3),
 ('World', 3),
 ('Hip Hop/Rap', 2),
 ('Sci Fi & Fantasy', 2),
 ('Drama', 2),
 ('Rock And Roll', 1),
 ('Bossa Nova', 1),
 ('Easy Listening', 1),
 ('Heavy Metal', 1),
 ('Science Fiction', 1),
 ('Comedy', 1),
 ('Opera', 1)]

In [79]:
query_all_genres = """
    SELECT
        genres.Name AS genre_name,
        COUNT(DISTINCT artists.ArtistId) AS num_artists
    FROM
        genres
    LEFT JOIN
        tracks ON genres.GenreId = tracks.GenreId
    LEFT JOIN
        albums ON tracks.AlbumId = albums.AlbumId
    LEFT JOIN
        artists ON albums.ArtistId = artists.ArtistId
    GROUP BY
        genres.GenreId, genre_name
    ORDER BY
        num_artists DESC;
"""

cursor.execute(query_all_genres)

all_genres_artists = cursor.fetchall()
all_genres_artists


[('Classical', 66),
 ('Rock', 51),
 ('Latin', 28),
 ('Alternative & Punk', 16),
 ('Metal', 14),
 ('Jazz', 10),
 ('Blues', 5),
 ('TV Shows', 5),
 ('Alternative', 5),
 ('Soundtrack', 4),
 ('R&B/Soul', 4),
 ('Reggae', 3),
 ('Pop', 3),
 ('Electronica/Dance', 3),
 ('World', 3),
 ('Hip Hop/Rap', 2),
 ('Sci Fi & Fantasy', 2),
 ('Drama', 2),
 ('Rock And Roll', 1),
 ('Bossa Nova', 1),
 ('Easy Listening', 1),
 ('Heavy Metal', 1),
 ('Science Fiction', 1),
 ('Comedy', 1),
 ('Opera', 1)]

In [80]:
genre_most_tracks = [genre[0] for genre in top_5_genres[:5]]
genre_most_tracks

['Rock', 'Latin', 'Metal', 'Alternative & Punk', 'Jazz']

In [81]:
genre_most_artists = [genre[0] for genre in all_genres_artists[:5]]
genre_most_artists

['Classical', 'Rock', 'Latin', 'Alternative & Punk', 'Metal']

In [82]:
# Create a dictionary for quick lookup of artists data
artists_dict = dict(all_genres_artists)

# Iterate over top_5_genres and update each entry with the number of artists
for i, (genre, track_count) in enumerate(top_5_genres):
    # Get the number of artists for the current genre or default to 0
    artist_count = artists_dict.get(genre, 0)
    
    # Update the entry with the number of artists
    top_5_genres[i] = (genre, track_count, artist_count)

# Print or use the updated list
print(top_5_genres)

[('Rock', 1297, 51), ('Latin', 579, 28), ('Metal', 374, 14), ('Alternative & Punk', 332, 16), ('Jazz', 130, 10), ('TV Shows', 93, 5), ('Blues', 81, 5), ('Classical', 74, 66), ('Drama', 64, 2), ('R&B/Soul', 61, 4), ('Reggae', 58, 3), ('Pop', 48, 3), ('Soundtrack', 43, 4), ('Alternative', 40, 5), ('Hip Hop/Rap', 35, 2), ('Electronica/Dance', 30, 3), ('Heavy Metal', 28, 1), ('World', 28, 3), ('Sci Fi & Fantasy', 26, 2), ('Easy Listening', 24, 1), ('Comedy', 17, 1), ('Bossa Nova', 15, 1), ('Science Fiction', 13, 1), ('Rock And Roll', 12, 1), ('Opera', 1, 1)]


In [83]:
json_data_genres = {"genres_data": []}

for genre in top_5_genres:
    json_data_genres["genres_data"].append({
        "genre_name": genre[0],  # Assuming the artist_id is at index 0
        "genre_tracks": genre[1],       # Assuming the name is at index 1
        "genre_artists": genre[2] # Assuming the album_count is at index 2
    })

path_correct_genre_data = "../data/processed_data/transformed_genre_data.json"

with open(path_correct_genre_data, "w") as file:
    json.dump(json_data_genres, file, indent=2)


In [84]:
genres_data_list = json_data_genres['genres_data']

genres_data_list

[{'genre_name': 'Rock', 'genre_tracks': 1297, 'genre_artists': 51},
 {'genre_name': 'Latin', 'genre_tracks': 579, 'genre_artists': 28},
 {'genre_name': 'Metal', 'genre_tracks': 374, 'genre_artists': 14},
 {'genre_name': 'Alternative & Punk',
  'genre_tracks': 332,
  'genre_artists': 16},
 {'genre_name': 'Jazz', 'genre_tracks': 130, 'genre_artists': 10},
 {'genre_name': 'TV Shows', 'genre_tracks': 93, 'genre_artists': 5},
 {'genre_name': 'Blues', 'genre_tracks': 81, 'genre_artists': 5},
 {'genre_name': 'Classical', 'genre_tracks': 74, 'genre_artists': 66},
 {'genre_name': 'Drama', 'genre_tracks': 64, 'genre_artists': 2},
 {'genre_name': 'R&B/Soul', 'genre_tracks': 61, 'genre_artists': 4},
 {'genre_name': 'Reggae', 'genre_tracks': 58, 'genre_artists': 3},
 {'genre_name': 'Pop', 'genre_tracks': 48, 'genre_artists': 3},
 {'genre_name': 'Soundtrack', 'genre_tracks': 43, 'genre_artists': 4},
 {'genre_name': 'Alternative', 'genre_tracks': 40, 'genre_artists': 5},
 {'genre_name': 'Hip Hop/Rap'

In [85]:
query = """
SELECT COUNT(CustomerId) AS num_customers
FROM customers;
"""

cursor.execute(query)
number_customers = list((cursor.fetchall())[0])
number_customers

[59]

In [86]:
likability_score = []

for genre in genres_data_list:
    score = round(((0.4 * genre['genre_tracks']) + (0.6 * genre['genre_artists']) / number_customers[0]), 2) 

    likability_score.append(score)

likability_score

[519.32,
 231.88,
 149.74,
 132.96,
 52.1,
 37.25,
 32.45,
 30.27,
 25.62,
 24.44,
 23.23,
 19.23,
 17.24,
 16.05,
 14.02,
 12.03,
 11.21,
 11.23,
 10.42,
 9.61,
 6.81,
 6.01,
 5.21,
 4.81,
 0.41]

In [87]:
json_data_genres = {"genres_data": []}

for genre, score in zip(top_5_genres, likability_score):
    json_data_genres["genres_data"].append({
        "genre_name": genre[0],
        "genre_tracks": genre[1],
        "genre_artists": genre[2],
        "genre_likability_score": score
    })

path_correct_genre_data = "../data/processed_data/newest_transformed_genre_data.json"

with open(path_correct_genre_data, "w") as file:
    json.dump(json_data_genres, file, indent=2)


In [88]:
query_mediatypes = "SELECT * FROM media_types;"

cursor.execute(query_mediatypes)
results_test = cursor.fetchall()
results_test

[(1, 'MPEG audio file'),
 (2, 'Protected AAC audio file'),
 (3, 'Protected MPEG-4 video file'),
 (4, 'Purchased AAC audio file'),
 (5, 'AAC audio file')]